# （一）背景描述

### 1、问题提出

在红酒的各项特征中，可以通过酒精浓度和红酒的颜色判断出红酒的年份，通过检测红酒中酚类化合物、黄酮类化合物成分判断红酒是否过期。因此我们可以通过对红酒各项成分含量的分析，来判断出红酒的品质，也就可以判断出红酒的所属类别。

判别红酒分类主要有感官评定方法和常规理化指标检测方法，虽然感官评定是目前国内外鉴定葡萄酒品质的主要手段，但由于该方法主要是评定人员通过红酒颜色强度、气味、色调和滋味等感官指标来对红酒进行分类，因此该方法具有较大的主观性。为了减少感官评定方法可能会产生的红酒分类错误，本例子采用深度学习中的全连接神经网络结构对红酒常规理化指标进行模型训练，并以学得的模型预测红酒所属类别，以提高红酒分类效率。

| 列名 | 说明 | 属性类型 |
| --- | --- | --- |
| class | 三个类别：class0，class1，class2 | 离散 |
| alcohol | 酒精浓度(%) | 连续 |
| malic_acid | 苹果酸 | 连续 |
| ash | 灰烬 | 连续 |
| alcalinity_of_ash | 灰分的碱度 | 连续 |
| magnesium | 镁(mg/L) | 连续 |
| total_phenols | 总酚(mg/L) | 连续 |
| flavanoids | 黄酮类化合物 | 连续 |
| nonflavanoid_phenols | 非黄烷类酚类 | 连续 |
| proanthocyanins | 原花色素 | 连续 |
| color_intensity | 颜色强度 | 连续 |
| hue | 色调 | 连续 |
| od280/od315_of_diluted_wines | 稀释葡萄酒的OD280/OD315 | 连续 |
| proline | 脯氨酸 | 连续 |

### 2、研究目的

基于红酒常规理化指标数据，训练学得一个可以预测红酒分类的全连接神经网络模型，并研究使用该模型进行红酒分类的准确率。

# （二）数据处理

### 1、数据来源

Wine葡萄酒数据集是来自UCI上面的公开数据集，该数据是对意大利在不同地点所生产的三种葡萄酒进行化学分析的结果。 由于sklearn自带的红酒数据集（wine）与UCI上面的公开数据集的数据内容一样，所以本实验直接通过sklearn.datasets导入红酒数据集。

In [ ]:
from sklearn.datasets import load_wine  #红酒数据

### 2、数据描述

In [ ]:
import pandas as pd
import numpy as np

wine = load_wine()
x = wine.data  #特征数据
y = wine.target  #标签数据
features_name = wine.feature_names
target_name = wine.target_names
print('特征名称：',features_name)
print('标签类别：',target_name)

df1 = pd.DataFrame(data=x, columns=features_name)  #将特征数据转化为Dataframe
df2 = pd.DataFrame(data=y, columns=['class'])  #将标签数据转化为Dataframe
df = pd.concat([df1,df2],axis=1)  #合并特征数据和标签数据
df

In [ ]:
df['class'].value_counts()

可以看到在wine数据集中，共有178个样本，每个样本有13个特征（前13列数据），最后一列为红酒所属类别（其中第1类有59个样本，第2类有71个样本，第3类有48个样本）。

In [ ]:
df.info()

### 3、数据预处理

数据分割：将80%的数据集作为训练集，其余的20%为测试集。

In [ ]:
from sklearn.model_selection import train_test_split

x_train0, x_test0, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2022)

归一化处理：通过调用sklearn库的标准化函数MinMaxScaler对数据进行归一化处理。

In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_train = min_max_scaler.fit_transform(x_train0)   # 使用了离差标准化方法进行归一化处理
x_test = min_max_scaler.fit_transform(x_test0)

转化数据为Tensor类型：

Tensor（张量）是PyTorch中重要的数据结构，是存储和变换数据的主要⼯具，并且Tensor和Numpy的多维数组⾮常类似，但Tensor可以使用GPU进行加速计算。因为本实验是基于PyTorch框架实现红酒分类，Tensor是深度学习中核心的数据结构，所以需要将数据转换为Tensor类型。

接下来我们需要对训练和测试的特征标签转换为Tensor格式，默认的Tensor是数据类型是FloatTensor。
- 由上面数据初探可以发现，红酒数据的特征的数据类型都是float64，所以我们将训练和测试的特征转为FloatTensor的32位浮点类型数据。
- 由于标签为类别为int32，如果按默认格式转为浮点型数据下面运行会报错，所以我们将训练和测试的标签转为LongTensor的64位整型数据。

In [ ]:
import torch
import torch.utils.data as data

# 将数据转换为tensor类型
x_train = torch.FloatTensor(x_train)  
y_train = torch.LongTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)
print(y_test) 

# （三）模型构建

通过创建一个类，定义全连接神经网络
+  构建子模块：在自己建立的模型（继承torch.nn.Module）的__init__()方法
+  拼接子模块：在模型的forward()方法中

本实验使用的是两层的全连接神经网络模型，采用relu()函数作为隐藏层的激活函数，采用sigmoid()函数作为输出层的激活函数

### 1、设计全连接神经网络的结构

In [ ]:
import torch.nn as nn

# 建立
class NetModel(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(NetModel, self).__init__()
        self.hiddden = torch.nn.Linear(n_feature, n_hidden, bias = True)  # 定义隐层网络
        self.out = torch.nn.Linear(n_hidden, n_output, bias = True)  # 定义输出层网络
        self.relu = nn.ReLU()   # 定义relu激活函数
        #self.dropout = nn.Dropout(p=0.5)   # 定义dropout正则化，以概率0.5随机的将参数置0
        self.sigmoid = nn.Sigmoid()   # 定义sigmoid激活函数
        
    # forward()是实现网络中不同层的连接关系（即前向传播）
    def forward(self, x):
        x = self.hiddden(x)
        x = self.relu(x)  # 隐藏层激活函数采用relu()函数
        #x = self.dropout(x)   # 如果使用了dropout正则化，这里需要删除注释符号
        x = self.out(x)
        x = self.sigmoid(x)   # 预测值
        return x

# （四）训练模型

### 1、定义模型训练函数

In [ ]:
import matplotlib.pyplot as plt   # 函数定义中存在绘制每次迭代损失值的图像，因此需要导入该模块
import numpy as np
#配置中文显示
plt.rcParams['font.family'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False  

# 参数的解释
def fit(net, loader, lr, epochs):
    # 对于多分类一般使用交叉熵损失函数
    loss_fun = torch.nn.CrossEntropyLoss()
    # 使用Adam优化器
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    
    running_loss = []   # 用于保存每次迭代的loss值
    for i in range(epochs):
        print("---------第{}轮训练开始------------".format((i + 1)))
        train_loss = 0.0   # 用于汇总每批次计算出来的loss值
        train_acc = 0.0   # 用于汇总每批次计算出来正确的预测数量
        
        for step, (batch_x, batch_y) in enumerate(loader):
            
            output = net.forward(batch_x)   # 执行前向传播
            train_acc += (output.max(1)[1] == batch_y).sum().item()
            loss = loss_fun(output, batch_y)   # 计算损失值
            
            loss.backward()   # 反向传播
            optimizer.step()   # 更新梯度
            optimizer.zero_grad()   # 梯度清零
            
            train_loss += loss.item()            
            # 每到10个批次时（因为当前样本量较少，当样本量较多时可以适当增大这个值）或到达最后一个批次时，输出一次当前loss值
            if (step + 1) % 10 == 0 or step == (len(loader) - 1):
                print('迭代次数：%d, batch数：%5d 当前loss值: %.3f' % (i + 1, np.ceil((step + 1)/10), train_loss))
        
        print('第%d次完整迭代的损失值：%.3f' % (i + 1, train_loss))
        train_acc = train_acc/len(x_train) * 100
        print('第%d次完整迭代的正确率：%.1f%%' % (i + 1, train_acc))
        
        running_loss.append(train_loss)
    
    # 绘制损失变化图
    plt.plot(np.arange(epochs), running_loss, color = 'r', alpha = 0.3)
    plt.xticks(np.arange(0, epochs, 5))
    plt.title('训练集损失函数值随迭代次数的变化趋势')
    plt.xlabel('epoches')
    plt.ylabel('train_loss')

### 2、定义模型测试函数

In [ ]:
def testNN(net, x_test, y_test):
    # 定义损失函数
    loss_fun = torch.nn.CrossEntropyLoss()

    with torch.no_grad():   # 模型在测试集中不需要进行反向传播
        test_acc = 0.0   #辅助计算测试集的准确率
        output = net.forward(x_test)
        test_acc += (output.max(1)[1] == y_test).sum().item()
        loss = loss_fun(output, y_test)   # 计算损失值

        print('测试集的损失值：%.3f' % (loss.item()))
        print('测试集的正确率：%.1f%%' % (test_acc/len(x_test) * 100))

### 3、开始训练模型

In [ ]:
batch_size = len(x_train)   # 梯度下降法一组的样本数量，这里使用了训练集整个样本集，即为全批量梯度下降法
loader = data.DataLoader(
    dataset=data.TensorDataset(x_train, y_train),
    batch_size=batch_size,   # 若更改batch_size为1到len(x_train)之间，即为mini-batch梯度下降法；若值为1，则为SGD梯度下降法
    shuffle=True   # 在每次迭代训练时是否将数据洗牌 
)

net = NetModel(13, 20, 3)
fit(net, loader=loader, lr=0.15, epochs=20)   # 学习率初始化为0.15

### 4、测试模型

In [ ]:
testNN(net, x_test, y_test)

# （五）模型评价

### 1、学习率的选择

In [ ]:
# 为了防止输出内容较多，对训练模型的函数里使用了print函数的代码进行注释
# 由于这一小节需要绘制不同alpha值下，成本函数值随迭代次数增加的变化趋势，因此我们还需要在训练模型函数的最后加入一行返回代码，返回不同迭代次数下的损失函数值

# 改写训练模型的函数，加多一个参数methods，表示使用哪一种优化算法进行模型训练
def fit_upd(net, loader, lr, epochs, methods='optAda'):   # 训练模型的函数默认使用Adam优化算法
    # 对于多分类一般使用交叉熵损失函数
    loss_fun = torch.nn.CrossEntropyLoss()
    
    if methods =='optMomentum':
        optimizer = torch.optim.SGD(net.parameters(), lr=lr)   # 动量梯度优化算法
    elif methods =='optRMSprop':
        optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)   # RMSprop优化算法
    else :
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)   # Adam优化算法
    
    running_loss = []   # 用于保存每次迭代的loss值
    for i in range(epochs):
        train_loss = 0.0   # 用于汇总每批次计算出来的loss值
        
        for step, (batch_x, batch_y) in enumerate(loader):
            output = net.forward(batch_x)   # 执行前向传播
            loss = loss_fun(output, batch_y)   # 计算损失值
            loss.backward()   # 反向传播
            optimizer.step()   # 更新梯度
            optimizer.zero_grad()   # 梯度清零
            train_loss += loss.item()            
        
        running_loss.append(train_loss)
    return running_loss

In [ ]:
LR = np.arange(0.09, 1, 0.2)   # 取多个alpha值，并观察不同alpha值下成本函数值随迭代次数增加的变化趋势

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.xlabel('epochs')
plt.ylabel('train_loss')
plt.title('不同alpha值在此网络模型中的训练速度比较')

for i in np.arange(len(LR)):
    net=NetModel(13, 20, 3)   # 构建神经网络模型
    loss = fit_upd(net, loader, lr=round(LR[i], 2), epochs=100)
    ax.plot(loss, label=round(LR[i], 2))
    plt.legend()

### 2、比较不同优化算法的收敛速度

In [ ]:
#  在学习率的选择一节中，可得出学习率在0.1左右时，模型可以较快地进行收敛，因此此处设置alpha=0.1
lr = 0.1
optimizers = ['optAda', 'optMomentum', 'optRMSprop']   # 不同的优化算法

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.xlabel('epochs')
plt.ylabel('train_loss')
plt.title('A comparation of the speed of different algrithoms')

for i in optimizers:
    net=NetModel(13, 20, 3)   # 构建神经网络模型
    loss = fit_upd(net, loader, lr=lr, epochs=100, methods=i)
    ax.plot(loss, label=i)
    plt.legend()

### 3、Droupout正则化的使用

在模型构建类中已定义了droupout正则化，同学们可通过删除注释符号观察正则化后的模型效果。

In [70]:
class S:
    v=0
    @property
    def func(self):
        self.var=S.v
        return 0


In [87]:
def decorator(f):
    def func(*args):
        print(args)
        print('this is decorated function')
        print('func0=decorator(func0)')
        f()
        return 'anything'
    return func

@decorator
def func0():
    print('this is formor function')

func0()

()
this is decorated function
func0=decorator(func0)
this is formor function


'anything'